In [ ]:
#| label: setup-topic-2-2
#| warning: false

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score

import warnings
warnings.filterwarnings('ignore')

# Set Plotly theme
pio.templates.default = "plotly_white"

print("✓ All libraries loaded successfully!")

## Data Loading and Exploration

In [ ]:
#| label: load-data-topic-2-2

# Load lightcast job postings data
df = pd.read_csv('data/lightcast_job_postings.csv')

print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nFirst few rows:")
df.head()

In [ ]:
#| label: data-inspection

# Check data quality
print("="*80)
print("DATA QUALITY ASSESSMENT")
print("="*80)

# Key columns for analysis
key_columns = ['SALARY', 'STATE', 'TITLE', 'NAICS_2022_2', 'SOC_2', 'ONET', 'LIGHTCAST_SECTORS']

info_df = pd.DataFrame({
    'Column': key_columns,
    'Missing': [df[col].isnull().sum() if col in df.columns else 'N/A' for col in key_columns],
    'Missing %': [f"{(df[col].isnull().sum() / len(df) * 100):.2f}%" if col in df.columns else 'N/A' for col in key_columns],
    'Unique Values': [df[col].nunique() if col in df.columns else 'N/A' for col in key_columns]
})

print(info_df.to_string(index=False))

# Salary statistics
if 'SALARY' in df.columns:
    print(f"\nSalary Statistics:")
    print(f"  Mean: ${df['SALARY'].mean():,.2f}")
    print(f"  Median: ${df['SALARY'].median():,.2f}")
    print(f"  Std Dev: ${df['SALARY'].std():,.2f}")
    print(f"  Range: ${df['SALARY'].min():,.2f} - ${df['SALARY'].max():,.2f}")

## Data Preprocessing

In [ ]:
#| label: preprocessing

# Create working copy
df_clean = df.copy()

# Remove rows with missing salary or state
required_cols = ['SALARY', 'STATE']
initial_rows = len(df_clean)
df_clean = df_clean.dropna(subset=required_cols)
removed_rows = initial_rows - len(df_clean)

print(f"Removed {removed_rows:,} rows with missing salary or state data")
print(f"Final dataset: {len(df_clean):,} rows")

In [ ]:
#| label: political-leaning-creation

# Create political leaning variable based on 2024 presidential election
red_states = [
    'AL', 'AK', 'AR', 'FL', 'ID', 'IN', 'IA', 'KS', 'KY', 
    'LA', 'MS', 'MO', 'MT', 'NE', 'ND', 'OH', 'OK', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'WV', 'WY'
]

blue_states = [
    'CA', 'CO', 'CT', 'DE', 'HI', 'IL', 'ME', 'MD', 'MA', 
    'MI', 'MN', 'NH', 'NJ', 'NM', 'NY', 'OR', 'PA', 'RI', 
    'VT', 'VA', 'WA', 'WI', 'DC'
]

swing_states = ['AZ', 'GA', 'NC', 'NV']

def assign_political_leaning(state):
    if pd.isna(state):
        return 'Unknown'
    state = str(state).strip().upper()
    if state in red_states:
        return 'Red'
    elif state in blue_states:
        return 'Blue'
    elif state in swing_states:
        return 'Swing'
    else:
        return 'Other'

df_clean['political_leaning'] = df_clean['STATE'].apply(assign_political_leaning)

print("\nPolitical Leaning Distribution:")
print(df_clean['political_leaning'].value_counts())
print(f"\nPercentage:")
print((df_clean['political_leaning'].value_counts() / len(df_clean) * 100).round(2))

In [ ]:
#| label: visualize-political-distribution

# Visualize political distribution
fig = px.pie(
    values=df_clean['political_leaning'].value_counts().values,
    names=df_clean['political_leaning'].value_counts().index,
    title='Distribution of Jobs by Political Leaning of State',
    hole=0.4,
    color_discrete_map={'Red': '#FF6B6B', 'Blue': '#4ECDC4', 'Swing': '#FFD93D', 'Other': '#95A5A6'}
)
fig.update_layout(template="plotly_white", height=400)
fig.show()

## K-Means Clustering (unsupervised)


In [ ]:
#| label: clustering-reference-label

# Determine which reference label to use
reference_label = None
for label in ['SOC_2', 'NAICS_2022_2', 'ONET', 'LIGHTCAST_SECTORS']:
    if label in df_clean.columns and df_clean[label].notna().sum() > 0:
        reference_label = label
        print(f"✓ Using {label} as reference label")
        break

if reference_label is None:
    print("Using TITLE as reference.")
    reference_label = 'TITLE'

print(f"\nReference Label: {reference_label}")
print(f"Unique values: {df_clean[reference_label].nunique():,}")
print(f"\nTop 10 {reference_label} categories:")
print(df_clean[reference_label].value_counts().head(10))